In [1]:
from google.colab import drive
drive.mount('/drive', force_remount=True)
!ln -s "/drive/MyDrive/LeafDisease" "/content/LeafDisease"

Mounted at /drive


In [11]:
%cd LeafDisease
!ls

/drive/MyDrive/LeafDisease
binary_classification  models		     plantdisease.zip
dataset		       multi_classification  PlantVillage


In [12]:

!unzip plantdisease.zip

Archive:  plantdisease.zip
replace PlantVillage/Pepper__bell___Bacterial_spot/0022d6b7-d47c-4ee2-ae9a-392a53f48647___JR_B.Spot 8964.JPG? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace PlantVillage/Pepper__bell___Bacterial_spot/006adb74-934f-448f-a14f-62181742127b___JR_B.Spot 3395.JPG? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace PlantVillage/Pepper__bell___Bacterial_spot/00f2e69a-1e56-412d-8a79-fdce794a17e4___JR_B.Spot 3132.JPG? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace PlantVillage/Pepper__bell___Bacterial_spot/01613cd0-d3cd-4e96-945c-a312002037bf___JR_B.Spot 3262.JPG? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace PlantVillage/Pepper__bell___Bacterial_spot/0169b9ac-07b9-4be1-8b85-da94481f05a4___NREC_B.Spot 9169.JPG? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace PlantVillage/Pepper__bell___Bacterial_spot/018e494e-d2eb-468b-9d02-40219d9f4921___JR_B.Spot 9045.JPG? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace PlantVillage/Pepper__bell___Bacterial_spot/01940b6d-7dea-4889-a

In [13]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
from torchvision.utils import make_grid


class PlantVillageDataset(Dataset):
    """
    Custom Dataset for PlantVillage folder structure with proper binary labeling.

    Args:
        root_dir (str): Path to PlantVillage parent folder
        label_type (str): 'binary' or 'multi'
        transform (callable): Optional transform to apply
    """
    def __init__(self, root_dir, label_type='binary', transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.label_type = label_type

        # Get all class folders
        self.raw_classes = sorted(os.listdir(root_dir))
        self.classes = self._process_class_names(self.raw_classes)

        # Create samples
        self.samples = []
        for class_idx, class_name in enumerate(self.raw_classes):
            class_dir = os.path.join(root_dir, class_name)
            if not os.path.isdir(class_dir):
                continue

            for img_name in os.listdir(class_dir):
                if img_name.lower().endswith(('.png', '.jpg', '.jpeg')):
                    img_path = os.path.join(class_dir, img_name)
                    binary_label = 0 if 'healthy' in class_name.lower() else 1
                    multi_label = class_idx
                    self.samples.append((img_path, binary_label, multi_label))

    def _process_class_names(self, raw_classes):
        """Process class names based on label_type"""
        if self.label_type == 'binary':
            return ["Healthy", "Unhealthy"]
        else:
            return [name.replace('___', ' ').replace('__', ' ')
                   for name in raw_classes]

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        img_path, binary_label, multi_label = self.samples[idx]
        label = binary_label if self.label_type == 'binary' else multi_label

        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)

        return image, label

    def get_class_names(self):
        return self.classes

# Define transforms (same as your implementation)
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.5),
    transforms.RandomAffine(0, shear=10, scale=(0.8,1.2)),
    transforms.RandomRotation(26),
    transforms.GaussianBlur(kernel_size=5, sigma=(0.1, 2.0)),
    transforms.ToTensor(),
    transforms.Normalize(mean=mean, std=std)
])

test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=mean, std=std)
])

# Create datasets
def create_datasets(root_dir, label_type='binary'):
    full_dataset = PlantVillageDataset(
        root_dir=root_dir,
        label_type=label_type,
        transform=train_transform  # Will be overridden for test set
    )

    # Split dataset (80% train, 20% test)
    train_size = int(0.8 * len(full_dataset))
    test_size = len(full_dataset) - train_size
    train_dataset, test_dataset = torch.utils.data.random_split(
        full_dataset, [train_size, test_size]
    )

    # Override transform for test set
    test_dataset.dataset.transform = test_transform

    return train_dataset, test_dataset, full_dataset.get_class_names()

# Create dataloaders
def create_dataloaders(root_dir, batch_size=32, label_type='binary'):
    train_ds, test_ds, class_names = create_datasets(root_dir, label_type)

    train_loader = DataLoader(
        train_ds,
        batch_size=batch_size,
        shuffle=True,
        num_workers=4,
        pin_memory=True
    )

    test_loader = DataLoader(
        test_ds,
        batch_size=batch_size*2,
        shuffle=False,
        num_workers=2,
        pin_memory=True
    )

    return {
        'train': train_loader,
        'test': test_loader
    }

# Visualization function (same as yours)
def show_batch(dl, class_names, nrow=8):
    """Plot images grid of single batch"""
    for images, labels in dl:
        fig, ax = plt.subplots(figsize=(16, 12))
        ax.set_xticks([])
        ax.set_yticks([])

        # Unnormalize
        images = images * torch.tensor(std).view(3, 1, 1) + torch.tensor(mean).view(3, 1, 1)
        images = torch.clamp(images, 0, 1)

        grid = make_grid(images, nrow=nrow)
        ax.imshow(grid.permute(1, 2, 0))

        # Add class labels
        if len(labels) > nrow:
            labels = labels[:nrow]
        title = '\n'.join([class_names[l] for l in labels])
        ax.set_title(title, loc='left', pad=20)

        plt.show()
        break


In [14]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using:', device)

Using: cuda


In [15]:
from collections import Counter
import torch

class WeightCalculator:
    @staticmethod
    def calculate_weights(dataset, mode='binary'):
      """Calculate weights with safety checks"""
      if hasattr(dataset, 'samples'):
          labels = [multi_label for _, _, multi_label in dataset.samples]
      else:
          labels = [label for _, label in dataset]

      counts = Counter(labels)
      total = sum(counts.values())
      num_classes = len(counts)

      # Safe weight calculation with clamping
      weights_list = []
      for class_idx in range(num_classes):
          count = counts.get(class_idx, 1)  # Default to 1 if class missing
          if count == 0:
              weight = 1.0  # Avoid division by zero
          else:
              weight = total / (num_classes * count)
          # Clamp to reasonable values
          weight = max(0.1, min(weight, 10.0))
          weights_list.append(weight)

      return torch.tensor(weights_list, dtype=torch.float32)


In [24]:
full_dataset = PlantVillageDataset(
    root_dir="./PlantVillage",
    label_type='multi',
    transform=train_transform
)
dataloaders = create_dataloaders(root_dir="./PlantVillage", label_type='multi')
train_dl = dataloaders['train']
test_dl = dataloaders['test']

datasets = create_datasets(root_dir="./PlantVillage", label_type='multi')
train_data, test_data, class_names = datasets

print(len(train_data), len(test_data))
weights = WeightCalculator.calculate_weights(full_dataset, mode='multiclass')
num_classes = len(full_dataset.classes)
class_names = full_dataset.classes

9108 2277


In [17]:
%run "/content/LeafDisease/models/densenet121.py"
%run "/content/LeafDisease/models/efficientnetb0.py"
%run "/content/LeafDisease/models/mobilenetv2.py"
%run "/content/LeafDisease/models/leafnetv2.py"
%run "/content/LeafDisease/models/leafnet.py"
%run "/content/LeafDisease/models/resnet50.py"

cuda


In [18]:
import torch.nn as nn
import torch.nn.functional as F

In [19]:
weights = weights.to(device)  # Move weights to device

In [20]:
num_classes = len(full_dataset.classes)

models_info = []

# 1. ResNet
resnet = ResNetClassifier(num_classes=num_classes, weights=weights, freeze_backbone=True)
models_info.append(("ResNet50", resnet.get_model(), resnet.get_criterion(), resnet.get_optimizer()))

# 2. EfficientNet
effnet = EfficientNetClassifier(num_classes=num_classes, weights=weights, freeze_backbone=True)
models_info.append(("EfficientNetB0", effnet.get_model(), effnet.get_criterion(), effnet.get_optimizer()))

# 3. MobileNetV2
mobilenet = MobileNetV2Classifier(num_classes=num_classes, weights=weights, freeze_backbone=True)
models_info.append(("MobileNetV2", mobilenet.get_model(), mobilenet.get_criterion(), mobilenet.get_optimizer()))

# 4. DenseNet
densenet = DenseNet121Classifier(num_classes=num_classes, weights=weights, freeze_backbone=True)
models_info.append(("DenseNet121", densenet.get_model(), densenet.get_criterion(), densenet.get_optimizer()))

# 5. LeafNet (with weights!)
leafnet = LeafNet(n_class=num_classes)
leafnet = leafnet.to(device)
criterion_leaf = nn.CrossEntropyLoss(weight=weights) if weights is not None else nn.CrossEntropyLoss()
trainable_params = filter(lambda p: p.requires_grad, leafnet.parameters())
optimizer_leaf = torch.optim.Adam(trainable_params, lr=1e-4)
models_info.append(("LeafNet", leafnet, criterion_leaf, optimizer_leaf))

# 6. LeafNetv2 (with weights!)
leafnet2 = LeafNetv2(n_class=num_classes)
leafnet2 = leafnet2.to(device)
criterion_leaf2 = nn.CrossEntropyLoss(weight=weights) if weights is not None else nn.CrossEntropyLoss()
trainable_params2 = filter(lambda p: p.requires_grad, leafnet2.parameters())
optimizer_leaf2 = torch.optim.Adam(trainable_params2, lr=1e-4)
models_info.append(("LeafNetv2", leafnet2, criterion_leaf2, optimizer_leaf2))

/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


100%|██████████| 97.8M/97.8M [00:00<00:00, 191MB/s]
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B0_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Downloading: "https://download.pytorch.org/models/efficientnet_b0_rwightman-7f5810bc.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b0_rwightman-7f5810bc.pth


100%|██████████| 20.5M/20.5M [00:00<00:00, 214MB/s]
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


...

100%|██████████| 30.8M/30.8M [00:00<00:00, 208MB/s]
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth


100%|██████████| 528M/528M [00:02<00:00, 233MB/s]
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/476M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/28.8M [00:00<?, ?B/s]

In [21]:
%run "/content/LeafDisease/multi_classification/train.ipynb"


Drive already mounted at /drive; to attempt to forcibly remount, call drive.mount("/drive", force_remount=True).
binary_classification  LeafDisease  multi_classification  PlantVillage
dataset		       models	    plantdisease.zip


In [22]:
trainer = MultiClassClassifierTrainer(
    device=device,
    class_names=class_names
)

In [26]:
all_histories = {}
trained_models = []

for name, model, criterion, optimizer in models_info:
    print(f"\n===== Training {name} =====")
    print(f"\n traindata len: {len(train_data)}")
    print(f"\n testdata len: {len(test_data)}")
    model, history, y_true, y_pred = trainer.train(
        model, nn.CrossEntropyLoss(), optimizer,
        {'train':train_dl, 'val':test_dl}, {"train":train_data, "val":test_data},
        num_epochs=20, patience=5,
        save_path=f"{name}_history.pkl"
    )


    all_histories[name] = history
    trained_models.append((name, model))

    # Generate plots (in correct order)
    trainer.plot_history(history, model_name=name)
    trainer.eval_plot(y_true, y_pred, model_name=name)
    trainer.plot_roc(model, test_dl, model_name=name)
    trainer.plot_classification_report(y_true, y_pred, model_name=name)
trainer.plot_f1_curves(all_histories)
trainer.compare_models(trained_models, test_dl)


Output hidden; open in https://colab.research.google.com to view.